In [ ]:
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GenerationConfig

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = GPT2LMHeadModel.from_pretrained("out_rlhf").eval().to("cuda")

In [ ]:
dataset = load_dataset("CarperAI/openai_summarize_tldr", split="test")

In [ ]:
generation_config = GenerationConfig(
    max_new_tokens=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
example = dataset[500]
tokenized_prompt = tokenizer(example["prompt"], return_tensors="pt").to("cuda")

res = model.generate(
    **tokenized_prompt,
    generation_config=generation_config,
)

string_to_write = (
    "Query: \n"
    f"{example['prompt']}\n"
    f"Generated: \n"
    f"{tokenizer.decode(res[0, len(tokenized_prompt['input_ids'][0]):].cpu().numpy().tolist())}\n"
    f"True label: \n"
    f"{example['label']}\n"
    f"====================================================\n"
)
print(string_to_write)